In [10]:
from pydrake.all import (AddDefaultVisualization, AddMultibodyPlantSceneGraph,
                         ConstantVectorSource, DiagramBuilder,
                         DiscreteContactSolver, ModelVisualizer, PackageMap,
                         Parser, Simulator, StartMeshcat, namedview)

In [11]:
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7000


Note: The models are also available directly here, if that's easier: https://github.com/RussTedrake/kinova-movo/tree/movo_description_drake

In [12]:
def setup_package_map(parser):
    parser.package_map().AddRemote(
        package_name="kinova-movo",
        params=PackageMap.RemoteParams(
            urls=[
                "https://github.com/RussTedrake/kinova-movo/archive/d94d1d7da7ff8fc71f2439bb0a8989f1e6fd79b4.tar.gz"
            ],
            sha256="a9201477a23f410f10d00e86847de778c175d3d3c8971be52a9ac881194e4887",
            strip_prefix="kinova-movo-d94d1d7da7ff8fc71f2439bb0a8989f1e6fd79b4",
        ))
    parser.package_map().AddPackageXml(f'{parser.package_map().GetPath("kinova-movo")}/movo_common/movo_description/package.xml')
'''
def setup_package_map(parser):
    parser.package_map().AddPackageXml('/home/russt/tmp/kinova-movo/movo_common/movo_description/package.xml')
    parser.package_map().AddPackageXml('/home/russt/QR/assets/package.xml')
    parser.package_map().AddPackageXml(
        '/home/russt/QR/assets/open-world-tamp/package.xml')
'''

"\ndef setup_package_map(parser):\n    parser.package_map().AddPackageXml('/home/russt/tmp/kinova-movo/movo_common/movo_description/package.xml')\n    parser.package_map().AddPackageXml('/home/russt/QR/assets/package.xml')\n    parser.package_map().AddPackageXml(\n        '/home/russt/QR/assets/open-world-tamp/package.xml')\n"

# First attempt -- SAP w/ urdf mimic joints => coupler constraints.

In [13]:
with_mimic = """
directives:
- add_model:
    name: robotiq
    file: package://movo_description/urdf/manipulation/robotiq/robotiq_85_gripper.urdf

- add_weld:
    parent: world
    child: robotiq::robotiq_coupler_link
    X_PC:
        rotation: !Rpy { deg: [90.0, 0.0, -90.0 ]}

- add_model:
    name: spam
    file: package://qr_assets/010_potted_meat_can_hydro.sdf
    default_free_body_pose: { base_link_meat: { 
        translation: [0.2, 0.0, 0.0],
        rotation: !Rpy { deg: [-90.0, 0.0, 0.0 ]}
    } }

- add_model:
    name: table
    file: package://OpenWorldTAMP/models/table.urdf
    
- add_weld:
    parent: world
    child: table::base
    X_PC:
        translation: [0.0, 0.0, -0.775]
"""

v = ModelVisualizer(meshcat=meshcat)
# TODO(russt): Grab movo_description as a remote package.
setup_package_map(v.parser())
v.parser().AddModelsFromString(with_mimic, "dmd.yaml")
v.Run(loop_once=True)
meshcat.SetProperty("proximity", "visible", True)

RuntimeError: error: URI 'package://qr_assets/010_potted_meat_can_hydro.sdf' refers to unknown package 'qr_assets'

In [ ]:
def sim_w_mimic():
    builder = DiagramBuilder()
    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.002)
    plant.set_discrete_contact_solver(DiscreteContactSolver.kSap)
    parser = Parser(plant)
    setup_package_map(parser)
    parser.AddModelsFromString(with_mimic, "dmd.yaml")
    plant.Finalize()

    torque = builder.AddSystem(ConstantVectorSource([1]))
    builder.Connect(torque.get_output_port(), plant.get_actuation_input_port())

    AddDefaultVisualization(builder, meshcat=meshcat)

    diagram = builder.Build()
    simulator = Simulator(diagram)

    meshcat.StartRecording(set_visualizations_while_recording=False)
    simulator.AdvanceTo(10)
    meshcat.PublishRecording()

sim_w_mimic()

# Second attempt -- Using TAMSI + a Linear bushing element

Note that the simulation timestep is much smaller.  The contact forces are still jumping all over.

In [ ]:
with_bushing = """
directives:
- add_model:
    name: robotiq
    file: package://movo_description/sdf/manipulation/robotiq_woven/robotiq_85.sdf
    default_joint_positions:
        l_finger_joint: [0.59]
        l_tip_joint: [-0.59]
        r_finger_joint: [-0.59]
        r_tip_joint: [0.59]

- add_weld:
    parent: world
    child: robotiq::hand_base_link

- add_model:
    name: spam
    file: package://drake/manipulation/models/ycb/sdf/010_potted_meat_can.sdf
    default_free_body_pose: { base_link_meat: { 
        translation: [0.15, 0.0, 0.0],
        rotation: !Rpy { deg: [-90.0, 0.0, 0.0 ]}
    } }

- add_model:
    name: table
    file: package://drake/examples/kuka_iiwa_arm/models/table/extra_heavy_duty_table_surface_only_collision.sdf
    
- add_weld:
    parent: world
    child: table::link
    X_PC:
        translation: [0.0, 0.0, -0.81]
"""

v = ModelVisualizer(meshcat=meshcat)
# TODO(russt): Grab movo_description as a remote package.
setup_package_map(v.parser())
v.parser().package_map().AddPackageXml(
    '/home/russt/tmp/movo_description_ws/src/movo_description/package.xml')
v.parser().AddModelsFromString(with_bushing, "dmd.yaml")
v.Run(loop_once=True)

In [ ]:
def sim_w_bushing():
    builder = DiagramBuilder()
    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=1e-4)
    plant.set_discrete_contact_solver(DiscreteContactSolver.kSap)
    parser = Parser(plant)
    setup_package_map(parser)
    parser.AddModelsFromString(with_bushing, "dmd.yaml")

    plant.Finalize()

    ActuatorView = namedview("actuators", plant.GetActuatorNames())
    torque = ActuatorView.Zero()
    torque.robotiq_l_actuated_joint = -1
    torque.robotiq_r_actuated_joint = 1
    torque_source = builder.AddSystem(ConstantVectorSource(torque))
    builder.Connect(torque_source.get_output_port(),
                    plant.get_actuation_input_port())

    AddDefaultVisualization(builder, meshcat=meshcat)

    diagram = builder.Build()
    simulator = Simulator(diagram)

    simulator.AdvanceTo(20)


sim_w_bushing()